Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### introduce  L2 regularization for logistic models 

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
reg = 0.01
batch_size = 128
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
        + reg*(tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
    )
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 47.182831
Minibatch accuracy: 13.3%
Validation accuracy: 16.7%
Test accuracy: 16.8%
Minibatch loss at step 500: 0.848063
Minibatch accuracy: 83.6%
Validation accuracy: 81.5%
Test accuracy: 87.1%
Minibatch loss at step 1000: 0.904085
Minibatch accuracy: 77.3%
Validation accuracy: 81.2%
Test accuracy: 86.9%
Minibatch loss at step 1500: 0.692767
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Test accuracy: 87.7%
Minibatch loss at step 2000: 0.781577
Minibatch accuracy: 82.0%
Validation accuracy: 82.1%
Test accuracy: 87.8%
Minibatch loss at step 2500: 0.739436
Minibatch accuracy: 85.2%
Validation accuracy: 81.8%
Test accuracy: 87.3%
Minibatch loss at step 3000: 0.784747
Minibatch accuracy: 83.6%
Validation accuracy: 80.9%
Test accuracy: 87.4%


### introduce  L2 regularization for NNet

In [7]:
batch_size = 128
hidden1 = 1024
graph = tf.Graph()
#define a forward function 
def forward(tf_X,W1,b1,W2,b2):
    Z1 = tf.nn.relu(tf.matmul(tf_X,W1) + b1)
    return tf.matmul(Z1,W2) + b2

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #weights 
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden1])
    )
    b1 = tf.Variable(tf.zeros([hidden1]))
    W2 = tf.Variable(
        tf.truncated_normal([hidden1,num_labels])
    )
    b2 = tf.Variable(tf.zeros([num_labels]))
    # Training computation
    logits_train = forward(tf_train_dataset,W1,b1,W2,b2)
    logits_valid = forward(tf_valid_dataset,W1,b1,W2,b2)
    logits_test = forward(tf_test_dataset,W1,b1,W2,b2)
    cost = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits_train)
    reg_cost = tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b2)
    loss = tf.reduce_mean(cost + reg*reg_cost)
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits_train)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [8]:
num_steps = 10000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3584.409424
Minibatch accuracy: 3.9%
Validation accuracy: 27.3%
Test accuracy: 29.7%
Minibatch loss at step 500: 21.135113
Minibatch accuracy: 88.3%
Validation accuracy: 84.8%
Test accuracy: 90.6%
Minibatch loss at step 1000: 0.962792
Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Test accuracy: 89.6%
Minibatch loss at step 1500: 0.704695
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Test accuracy: 90.0%
Minibatch loss at step 2000: 0.682201
Minibatch accuracy: 85.9%
Validation accuracy: 84.2%
Test accuracy: 90.0%
Minibatch loss at step 2500: 0.681856
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Test accuracy: 89.2%
Minibatch loss at step 3000: 0.713703
Minibatch accuracy: 86.7%
Validation accuracy: 83.5%
Test accuracy: 89.7%
Minibatch loss at step 3500: 0.713135
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Test accuracy: 89.5%
Minibatch loss at step 4000: 0.749180
Minibatch accuracy: 83.6%
Validation accuracy: 84.5%
Te

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
batch_size = 5000
hidden1 = 1024
graph = tf.Graph()
#define a forward function 
def forward(tf_X,W1,b1,W2,b2):
    Z1 = tf.nn.relu(tf.matmul(tf_X,W1) + b1)
    return tf.matmul(Z1,W2) + b2

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #weights 
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden1])
    )
    b1 = tf.Variable(tf.zeros([hidden1]))
    W2 = tf.Variable(
        tf.truncated_normal([hidden1,num_labels])
    )
    b2 = tf.Variable(tf.zeros([num_labels]))
    # Training computation
    logits_train = forward(tf_train_dataset,W1,b1,W2,b2)
    logits_valid = forward(tf_valid_dataset,W1,b1,W2,b2)
    logits_test = forward(tf_test_dataset,W1,b1,W2,b2)
    cost = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits_train)
    reg_cost = tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b2)
    loss = tf.reduce_mean(cost + reg*reg_cost)
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits_train)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [12]:
num_steps = 10000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3517.466309
Minibatch accuracy: 6.3%
Validation accuracy: 37.1%
Test accuracy: 39.5%
Minibatch loss at step 500: 21.155981
Minibatch accuracy: 86.6%
Validation accuracy: 86.9%
Test accuracy: 92.7%
Minibatch loss at step 1000: 0.863471
Minibatch accuracy: 83.9%
Validation accuracy: 85.7%
Test accuracy: 91.2%
Minibatch loss at step 1500: 0.694955
Minibatch accuracy: 84.3%
Validation accuracy: 85.8%
Test accuracy: 91.2%
Minibatch loss at step 2000: 0.671810
Minibatch accuracy: 85.6%
Validation accuracy: 85.7%
Test accuracy: 91.2%
Minibatch loss at step 2500: 0.695988
Minibatch accuracy: 85.1%
Validation accuracy: 85.7%
Test accuracy: 91.3%
Minibatch loss at step 3000: 0.681289
Minibatch accuracy: 85.4%
Validation accuracy: 85.7%
Test accuracy: 91.2%
Minibatch loss at step 3500: 0.703297
Minibatch accuracy: 84.9%
Validation accuracy: 85.8%
Test accuracy: 91.2%
Minibatch loss at step 4000: 0.680086
Minibatch accuracy: 85.1%
Validation accuracy: 85.9%
Te

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
batch_size = 5000
hidden1 = 1024
graph = tf.Graph()
#define a forward function 
def forward_train(tf_X,W1,b1,W2,b2):
    Z1 = tf.nn.relu(tf.matmul(tf_X,W1) + b1)
    Z2 = tf.matmul(Z1,W2) + b2
    return tf.nn.dropout(Z2,0.5)
def forward_predict(tf_X,W1,b1,W2,b2):
    Z1 = tf.nn.relu(tf.matmul(tf_X,W1) + b1)
    return tf.matmul(Z1,W2) + b2

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #weights 
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden1])
    )
    b1 = tf.Variable(tf.zeros([hidden1]))
    W2 = tf.Variable(
        tf.truncated_normal([hidden1,num_labels])
    )
    b2 = tf.Variable(tf.zeros([num_labels]))
    # Training computation
    logits = forward_train(tf_train_dataset,W1,b1,W2,b2)
    logits_train = forward_predict(tf_train_dataset,W1,b1,W2,b2)
    logits_valid = forward_predict(tf_valid_dataset,W1,b1,W2,b2)
    logits_test = forward_predict(tf_test_dataset,W1,b1,W2,b2)
    cost = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits)
    reg_cost = tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b2)
    loss = tf.reduce_mean(cost + reg*reg_cost)
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits_train)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [17]:
num_steps = 5000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3772.545654
Minibatch accuracy: 14.4%
Validation accuracy: 34.2%
Test accuracy: 36.7%
Minibatch loss at step 500: 21.772900
Minibatch accuracy: 86.7%
Validation accuracy: 86.6%
Test accuracy: 92.5%
Minibatch loss at step 1000: 1.469686
Minibatch accuracy: 83.9%
Validation accuracy: 85.5%
Test accuracy: 91.1%
Minibatch loss at step 1500: 1.334017
Minibatch accuracy: 84.3%
Validation accuracy: 85.5%
Test accuracy: 91.0%
Minibatch loss at step 2000: 1.324668
Minibatch accuracy: 85.5%
Validation accuracy: 85.4%
Test accuracy: 91.1%
Minibatch loss at step 2500: 1.336433
Minibatch accuracy: 84.7%
Validation accuracy: 85.7%
Test accuracy: 91.1%
Minibatch loss at step 3000: 1.320215
Minibatch accuracy: 85.3%
Validation accuracy: 85.6%
Test accuracy: 91.2%
Minibatch loss at step 3500: 1.334157
Minibatch accuracy: 84.8%
Validation accuracy: 85.7%
Test accuracy: 91.0%
Minibatch loss at step 4000: 1.357705
Minibatch accuracy: 84.9%
Validation accuracy: 85.8%
T

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [22]:
batch_size = 5000
hidden1 = 1024
graph = tf.Graph()
#define a forward function 
def forward_train(tf_X,W1,b1,W2,b2):
    Z1 = tf.nn.relu(tf.matmul(tf_X,W1) + b1)
    Z2 = tf.matmul(Z1,W2) + b2
    return tf.nn.dropout(Z2,0.5)
def forward_predict(tf_X,W1,b1,W2,b2):
    Z1 = tf.nn.relu(tf.matmul(tf_X,W1) + b1)
    return tf.matmul(Z1,W2) + b2

with graph.as_default():
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #weights 
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden1])
    )
    b1 = tf.Variable(tf.zeros([hidden1]))
    W2 = tf.Variable(
        tf.truncated_normal([hidden1,num_labels])
    )
    b2 = tf.Variable(tf.zeros([num_labels]))
    # Training computation
    logits = forward_train(tf_train_dataset,W1,b1,W2,b2)
    logits_train = forward_predict(tf_train_dataset,W1,b1,W2,b2)
    logits_valid = forward_predict(tf_valid_dataset,W1,b1,W2,b2)
    logits_test = forward_predict(tf_test_dataset,W1,b1,W2,b2)
    cost = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits)
    reg_cost = tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b2)
    loss = tf.reduce_mean(cost + reg*reg_cost)
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    train_prediction = tf.nn.softmax(logits_train)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [23]:
num_steps = 2000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3735.712402
Minibatch accuracy: 8.3%
Validation accuracy: 30.2%
Test accuracy: 31.8%
Minibatch loss at step 500: 21.791349
Minibatch accuracy: 86.5%
Validation accuracy: 86.4%
Test accuracy: 92.3%
Minibatch loss at step 1000: 1.497170
Minibatch accuracy: 84.0%
Validation accuracy: 85.3%
Test accuracy: 91.0%
Minibatch loss at step 1500: 1.340338
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Test accuracy: 91.2%


### apply the RMS prop 

In [43]:
batch_size = 1000
hidden1 = 1024
graph = tf.Graph()
#define a forward function 
def forward_train(tf_X,W1,b1,W2,b2):
    Z1 = tf.nn.relu(tf.matmul(tf_X,W1) + b1)
    Z2 = tf.matmul(Z1,W2) + b2
    return tf.nn.dropout(Z2,0.5)
def forward_predict(tf_X,W1,b1,W2,b2):
    Z1 = tf.nn.relu(tf.matmul(tf_X,W1) + b1)
    return tf.matmul(Z1,W2) + b2

with graph.as_default():
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #weights 
    W1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden1])
    )
    b1 = tf.Variable(tf.zeros([hidden1]))
    W2 = tf.Variable(
        tf.truncated_normal([hidden1,num_labels])
    )
    b2 = tf.Variable(tf.zeros([num_labels]))
    # Training computation
    logits = forward_train(tf_train_dataset,W1,b1,W2,b2)
    logits_train = forward_predict(tf_train_dataset,W1,b1,W2,b2)
    logits_valid = forward_predict(tf_valid_dataset,W1,b1,W2,b2)
    logits_test = forward_predict(tf_test_dataset,W1,b1,W2,b2)
    cost = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits)
    reg_cost = tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b2)
    loss = tf.reduce_mean(cost + reg*reg_cost)
    # Optimizer
    optimizer = tf.train.RMSPropOptimizer(learning_rate=1e-4,decay=0.999,momentum=0.9).minimize(loss)
    train_prediction = tf.nn.softmax(logits_train)
    valid_prediction = tf.nn.softmax(logits_valid)
    test_prediction = tf.nn.softmax(logits_test)

In [44]:
num_steps = 10000
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3657.626465
Minibatch accuracy: 10.2%
Validation accuracy: 9.4%
Test accuracy: 9.2%
Minibatch loss at step 500: 3174.075928
Minibatch accuracy: 64.0%
Validation accuracy: 62.0%
Test accuracy: 68.1%
Minibatch loss at step 1000: 3114.821533
Minibatch accuracy: 66.3%
Validation accuracy: 70.2%
Test accuracy: 76.4%
Minibatch loss at step 1500: 3045.711426
Minibatch accuracy: 73.3%
Validation accuracy: 73.6%
Test accuracy: 79.9%
Minibatch loss at step 2000: 2963.413086
Minibatch accuracy: 74.3%
Validation accuracy: 75.4%
Test accuracy: 82.2%
Minibatch loss at step 2500: 2872.161621
Minibatch accuracy: 77.1%
Validation accuracy: 76.8%
Test accuracy: 83.5%
Minibatch loss at step 3000: 2752.740479
Minibatch accuracy: 76.8%
Validation accuracy: 77.9%
Test accuracy: 84.3%
Minibatch loss at step 3500: 2618.996094
Minibatch accuracy: 76.3%
Validation accuracy: 78.6%
Test accuracy: 85.2%
Minibatch loss at step 4000: 2448.225586
Minibatch accuracy: 77.1%
Validat